In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import figure_plotting as pl
import numpy as np

import os
import sys
from get_bpt import *
from plot_bpt import *
import bpt_processing as proc
import run_bpt as run

In [8]:
# Define plot parameters
plt.rcParams.update({'axes.spines.right':False,
                    'axes.spines.top':False,
                    'font.size': 14,
                    'axes.titlesize':20, # fontsize of the axes title
                    'axes.labelsize':20,
                    'axes.linewidth':2,
                    'legend.fontsize':18,
                    'xtick.labelsize':18,
                    'ytick.labelsize':18,
                    'lines.linewidth':2
                    })

In [ ]:
# Set data directory

# Download the data
url = 'https://drive.google.com/uc?id=1g3J-GC6oaxYquc296kdYaX5fLFgYM8ti'
output = 'data.zip'
gdown.download(url, output, quiet=False)
# Extract data to directory
with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall()


# Fig 1: Concept Figure

In [6]:
# Extract respiratory, cardiac, and head motion signals from BPT

# Plot respiratory, cardiac, and head motion in the same plot
pl.plot_motion_signals(inpdir="./concept", shift=-5)
plt.close()

# BPT Acq

In [7]:
# Plot params for this figure
plt.rcParams.update({'axes.spines.right':False,
                    'axes.spines.top':False,
                    'axes.labelsize':18,
                    'xtick.labelsize':16,
                    'lines.linewidth':2
                    })
# Input directory
inpdir = "./bpt_acq"

In [8]:
# Plot 3D ksp vs time and frequency graph
pl.plot_ksp_f(inpdir)
plt.close()

In [2]:
# Plot cardiac signals with overlaid purple points
pl.plot_cardiac_signals(inpdir="./bpt_acq")
plt.close()

# Rocker simulation

In [3]:
# Plot H-field magnitudes at 127.8MHz and 2.4/2.5278GHz
inpdir = "./sim/h_fields"
pl.plot_field_w_coils(inpdir, figsize=(16.7,5), xlim=[-45,35], ylim=[-35,35])
plt.close()

In [9]:
plt.rcParams.update({'axes.titlesize':18, # fontsize of the axes title
                    'figure.titlesize':18, # suptitle
                    'axes.labelsize':16,
                    'legend.fontsize':14,
                    'xtick.labelsize':14,
                    'ytick.labelsize':14,
                    'lines.linewidth':3
                    })

In [11]:
# Plot simulated BPT data
inpdir = "./sim/sim_bpt"
f_combined = pl.plot_sim(inpdir, xlims=[0,10], mirror=True, figsize=(12,5))
plt.close()

In [12]:
# Plot measured BPT data
inpdir = "./sim/meas_bpt"
pl.plot_meas_data(inpdir, f_combined, figsize=(12,5), c=np.array([9,8,10]))
plt.close()

# Rocker vibration

In [ ]:
def plot_vibration(pt_mag, t_starts, f_combined, period=12,
                   shifts=[-30, -50, -500, -100, -100, -100],
                   c_inds=np.arange(16,20),
                   figsize=(10,10), lw=2, cutoffs=[1,1,1,5,5,5]):
    # Plot
    if len(folder_list) > 3:
        nrows = 2
    else:
        nrows = 1
    fig, ax = plt.subplots(nrows=nrows, ncols=3, figsize=figsize)
    axs = ax.flatten()

    for i, folder in enumerate(folder_list):
        t_start = t_starts[i]
        t = np.arange(pt_obj.dims[1])*pt_obj.tr - t_start
        # Loop over coils
        for idx, c in enumerate(c_inds):
            pt = pt_mag[...,c]
            shift = shifts[i]
            line = axs[i].plot(t, (pt - np.mean(pt)) + shift*idx, label = "Coil {}".format(c-16), lw=lw)

        axs[i].set_xlim([0, period])
        
        axs[i].set_yticks([])
        axs[i].set_title(str(f_combined[i]) + "MHz")
        if i > 2 and nrows == 2:
            axs[i].set_xlabel("Time (s)")
        elif nrows == 1:
            axs[i].set_xlabel("Time (s)")
    plt.show()
    plt.savefig(fname + ".png", dpi=300, transparent=True)

    # Make legend
    lines = []
    labels = []
    for ax in fig.axes:
        Line, Label = ax.get_legend_handles_labels()
        lines.extend(Line)
        labels.extend(Label)
    legendfig = plt.figure()
    legendfig.legend(lines[:len(c_inds)], labels[:len(c_inds)], loc='center')
    plt.savefig(fname + "_legend.png", dpi=300, transparent=True)
    
    